## Image Folder

- torchvision.datasets.ImageFolder
    - User가 갖고 있는 Image를 폴더를 이용해 dataset 생성하는 방법
    - e.g.) root/Dogs와 root/Cats 

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision.datasets as datasets
import torchvision.transforms as transforms
import random
import numpy as np

from torch.utils.data.sampler import SubsetRandomSampler

device = torch.device("cuda:0")

# from PIL import Image, ImageFile
# ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
import torchvision
from torchvision import transforms

from torch.utils.data import DataLoader

In [3]:
from matplotlib.pyplot import imshow
%matplotlib inline

- transforms.Compose
    - transform을 여러 단계로 진행할 때 하나로 묶어 주는 것
    - Resize, ToTensor 등
    - transform.Normalize(mean, std), transform.RandomCrop(32, padding=4)

In [4]:
trans = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor()
])

total_data = torchvision.datasets.ImageFolder(root='./Dogs_and_Cats/PetImages/', transform=trans)

In [5]:
len(total_data)  # Dogs 12,000장, Cats 12,000장

24998

In [6]:
test_ratio = 0.1
batch_size = 32

num_data = len(total_data)
indices = list(range(num_data))
np.random.shuffle(indices)

split = int(np.floor(test_ratio * num_data))

In [7]:
train_idx, test_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
test_sampler = SubsetRandomSampler(test_idx)

In [8]:
# dataset loader
train_loader = DataLoader(dataset=total_data,
                          batch_size=batch_size,
                          sampler=train_sampler,
                          drop_last=True)
test_loader = DataLoader(dataset=total_data,
                          batch_size=batch_size,
                          sampler=test_sampler,
                          drop_last=True)

In [9]:
for X,Y in train_loader:
    print(X.shape)
    print(Y.shape)
    break

torch.Size([32, 3, 128, 128])
torch.Size([32])


In [10]:
class BasicCNN(nn.Module):
    
    def __init__(self):
        super(BasicCNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=5),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2),
        )        
        self.fc = nn.Sequential(
            nn.Linear(256*4*4, 512),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512, 128),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(128, 2),
        )
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0], -1)
        out = self.fc(out)
        return out

In [11]:
model = BasicCNN().to(device)

In [12]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss().to(device)

In [13]:
import warnings
warnings.filterwarnings("ignore")

In [14]:
total_batch = len(train_loader)
model.train()

for epoch in range(7):
    avg_loss=0 
    for itr, data in enumerate(train_loader):
        img, label = data
        img = img.to(device)
        label = label.to(device)        

        pred = model(img)
        loss = loss_fn(pred, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        avg_loss+=loss.item()
    print('Epoch:{} , loss:{:.5f}'.format(epoch, avg_loss/total_batch))
    
        

Epoch:0 , loss:0.67415
Epoch:1 , loss:0.61068
Epoch:2 , loss:0.54618
Epoch:3 , loss:0.49273
Epoch:4 , loss:0.43791
Epoch:5 , loss:0.38905
Epoch:6 , loss:0.34635


In [15]:
correct = 0
total = 0

with torch.no_grad():
    model.eval()
    for images, labels in test_loader:
        output = model(images.to(device))
        _, predicted = torch.max(output, dim=1)        
        total += labels.size(0)
        correct += (predicted ==labels.to(device)).sum().item()

print('Accuracy on test images: ', (correct / total))

Accuracy on test images:  0.8297275641025641
